In [1]:
################
# CHANGE BELOW #
################
repo = 'C:\\Users\\shepard\\Documents\\GitHub\\message_trade\\'

smip_scenarios = {'SSP1_baseline': {'model': 'SSP_SSP1_v2.3.1',
                                    'scenario': 'baseline',
                                    'gdx_file': 'MsgOutput_SSP_SSP1_v2.3_baseline.gdx'}}

import_from_gdx = False # If true, will draw on GDX files saved to a local directory.

In [12]:
#######################
# DO NOT CHANGE BELOW #
#######################
# Import required packages
import pandas as pd
import numpy as np
import openpyxl as oxl
from openpyxl.utils.dataframe import dataframe_to_rows
import csv
import os
import pickle

# Import MESSAGEix packages
import message_ix
import ixmp

# Import helper packages
os.chdir(repo + '\\code')
from check_packages import *
from dictionaries import *

# Pull in custom packages
os.chdir(repo + '\\code\\postprocess')
#from gdx_import import *
#from import_trade_gdx import *

# Directories
cwd = repo + 'data\\crosswalks\\'
gdxd = repo + 'data\\gdx\\'
iead = repo + 'data\\IEA\\WEB\\'
temp = repo + 'data\\temp\\'

In [3]:
mp = ixmp.Platform()

In [4]:
def import_trade_scenario(scenario, 
                          msg_model_name, 
                          msg_scenario_name,
                          trade_type,
                          mp_read = False,
                          mp = mp):
    
    # Import trade crosswalk based on imports vs. exports
    cw_trade = pd.read_excel(cwd + 'trade_activity.xlsx', sheet_name = trade_type)
    tec_list = list(cw_trade[cw_trade['group_type'] == 'tec']['group_name'])
    
    # Connect to ixmp if mp_read = True
    if mp_read == True: mp = ixmp.Platform()
        
    # Read scenario activity
    scen = message_ix.Scenario(mp, msg_model_name, msg_scenario_name)
    activity = scen.var('ACT',
                        filters = {'technology':tec_list})
    activity['SCENARIO'] = scenario
    activity['MODEL'] = msg_model_name
    
    # Merge trade crosswalk to activity to filter required technologies and obtain descriptors
    activity = activity.merge(cw_trade, left_on = 'technology', right_on = 'group_name', how = 'inner')
    
    # Define origin (if exports) or destination (if imports) based on MESSAGEix nodes
    if trade_type == 'Exports': nd = 'ORIGIN'
    elif trade_type == 'Imports': nd = 'DESTINATION'

    activity[nd] = ''
    for n in R12_nodes.keys():
        activity[nd] = np.where(activity['node_loc'] == 'R12_' + n, n, activity[nd])

    # Clean dataframe
    activity['YEAR'] = activity['year_act']
    activity = activity[['MODEL','SCENARIO', 'DESCRIPTION', 'UNIT', 'YEAR', 'FUEL', 'ORIGIN', 'DESTINATION', 'technology', 'lvl', 'mrg']]

    # Store both global pool levels ("total") and bilateral levels ("bilateral")
    if trade_type == 'Exports': gnd = 'DESTINATION'
    elif trade_type == 'Imports': gnd = 'ORIGIN'

    return {'total': activity[activity[gnd] == 'Global Pool'],
            'bilateral': activity[activity[gnd] != 'Global Pool']}

In [5]:
exports = dict(); imports = dict()
for s in smip_scenarios.keys():
    print('...importing ' + s)
    exports[s] =  import_trade_scenario(scenario = s,
                                        msg_model_name = smip_scenarios[s]['model'], 
                                        msg_scenario_name = smip_scenarios[s]['scenario'],
                                        trade_type = 'Exports')
    imports[s] =  import_trade_scenario(scenario = s,
                                        msg_model_name = smip_scenarios[s]['model'], 
                                        msg_scenario_name = smip_scenarios[s]['scenario'],
                                        trade_type = 'Imports')    

...importing SSP1_baseline


Existing index sets of 'map_tec_storage' ['node', 'technology', 'mode', 'technology', 'mode', 'level', 'commodity', 'lvl_temporal'] do not match ('node', 'technology', 'storage_tec', 'level', 'commodity')
Existing index sets of 'storage_initial' ['node', 'technology', 'mode', 'level', 'commodity', 'year', 'time'] do not match ('node', 'technology', 'level', 'commodity', 'year', 'time')
Existing index sets of 'storage_self_discharge' ['node', 'technology', 'mode', 'level', 'commodity', 'year', 'time'] do not match ('node', 'technology', 'level', 'commodity', 'year', 'time')
Existing index sets of 'map_tec_storage' ['node', 'technology', 'mode', 'technology', 'mode', 'level', 'commodity', 'lvl_temporal'] do not match ('node', 'technology', 'storage_tec', 'level', 'commodity')
Existing index sets of 'storage_initial' ['node', 'technology', 'mode', 'level', 'commodity', 'year', 'time'] do not match ('node', 'technology', 'level', 'commodity', 'year', 'time')
Existing index sets of 'storage

In [13]:
# Save pickles
exf = temp + 'exports.pkl'; imf = temp + 'imports.pkl'

with open(exf, 'wb') as f: pickle.dump(exports, f)
with open(imf, 'wb') as f: pickle.dump(imports, f)

In [14]:
mp.close_db()